- aspect ratios：高宽比率

假设 window 的尺寸为：$(w, h)$，锚框的尺寸为：$(w_1, h_1)$，则有：

$$
\begin{cases}
\frac{w_1 h_1}{wh} = s^2\\
\frac{h_1}{w_1} = \frac{h}{w} r
\end{cases}
$$

一般情况下，我们仅仅只需要考虑 $w=h=\text{base_size}$ 这种情况即可，我们可以有两种编程实现方式：

## 实现方式1

$$
\begin{cases}
w_s = \frac{w_1}{s} = \frac{w}{\sqrt{r}} = \text{round}(\sqrt{\frac{wh}{r}}) = \text{round}(\frac{\text{base_size}}{\sqrt{r}})\\
h_s = \frac{h_1}{s} = h \sqrt{r} = \sqrt{whr} = \text{round}(w_s r)
\end{cases}
$$

编程实现：

In [12]:
import numpy as np


class Anchor:
    def __init__(self, base_size):
        if not base_size:  # window 的大小
            raise ValueError("Invalid base_size: {}.".format(base_size))
        self._anchor = np.array([1, 1, base_size, base_size]) - 1  # 初始锚框的坐标

    @property
    def anchor(self):
        return self._anchor

    @anchor.setter
    def anchor(self, new_anchor):
        self._anchor = new_anchor

    @property
    def w(self):
        '''
        锚框的宽度
        '''
        return self.anchor[2] - self.anchor[0] + 1

    @property
    def h(self):
        '''
        锚框的高度
        '''
        return self.anchor[3] - self.anchor[1] + 1

    @property
    def size(self):
        '''
        锚框的面积
        '''
        return self.w * self.h

    @property
    def _whctrs(self):
        """
        Return x center, and y center for an anchor (window). 锚框的中心坐标
        """
        x_ctr = self.anchor[0] + 0.5 * (self.w - 1)
        y_ctr = self.anchor[1] + 0.5 * (self.h - 1)
        return np.array([x_ctr, y_ctr])

    @staticmethod
    def _coordinate(aspect, ctr):
        '''
        依据宽高组合计算锚框的坐标
        '''
        k = (aspect - 1) / 2
        return np.concatenate([ctr - k, ctr + k], axis=1)


class RPNAnchor(Anchor):
    def __init__(self, base_size, ratios, scales):
        super().__init__(base_size)
        if not isinstance(ratios, (tuple, list)):
            ratios = [ratios]
        if not isinstance(scales, (tuple, list)):
            scales = [scales]
        self.ratios = np.array(ratios)
        self.scales = np.array(scales)
        self._num_depth = len(self.ratios) * len(self.scales)  # 单个像素的锚框的个数
        self.base_anchors = self._base_anchors()

    @property
    def num_depth(self):
        """Number of anchors at each pixel."""
        return self._num_depth

    @property
    def ratio_aspects(self):
        '''
        依据 ratios 获取锚框的所有宽高组合
        '''
        size_ratios = self.size / self.ratios
        ws = np.round(np.sqrt(size_ratios))
        hs = np.round(ws * self.ratios)
        return np.stack([ws, hs], axis=1)

    @property
    def ratio_anchors(self):
        return self._coordinate(self.ratio_aspects, self._whctrs)

    @property
    def scale_aspects(self):
        '''
        依据 scales 获取锚框的所有宽高组合
        '''
        ws = self.w * self.scales
        hs = self.h * self.scales
        return np.stack([ws, hs], axis=1)

    @property
    def scale_anchors(self):
        return self._coordinate(self.scale_aspects, self._whctrs)

    def _base_anchors(self):
        '''
        获取 reference anchors
        '''
        anchors = []
        for anchor in self.ratio_anchors:
            self.anchor = anchor
            anchors.append(self.scale_anchors)
        return np.concatenate(anchors)

    def generator(self, alloc_size, stride):
        # propagete to all locations by shifting offsets
        height, width = alloc_size
        offset_x = np.arange(0, width * stride, stride)
        offset_y = np.arange(0, height * stride, stride)
        offset_x, offset_y = np.meshgrid(offset_x, offset_y)
        offsets = np.stack((offset_x.ravel(), offset_y.ravel(),
                            offset_x.ravel(), offset_y.ravel()), axis=1)
        # broadcast_add (1, N, 4) + (M, 1, 4)
        anchors = (self.base_anchors.reshape(
            (1, -1, 4)) + offsets.reshape((-1, 1, 4)))
        return anchors.reshape((1, 1, height, width, -1)).astype(np.float32)

In [13]:
scales = [8, 16, 32]  # 尺度，面积比
ratios = [0.5, 1, 2]  # window（滑动窗口） 与锚框的面积的比率（aspect ratios）
base_size = 16  # 滑动窗口的大小

self = RPNAnchor(base_size, ratios, scales)

In [14]:
self.ratio_anchors[0]

array([-3. ,  2.5, 18. , 12.5])

In [15]:
alloc_size = (50, 50)
stride = 16

In [16]:
self.generator(alloc_size, stride)

array([[[[[ -84.,  -40.,   99., ..., -344.,  183.,  359.],
          [ -68.,  -40.,  115., ..., -344.,  199.,  359.],
          [ -52.,  -40.,  131., ..., -344.,  215.,  359.],
          ...,
          [ 668.,  -40.,  851., ..., -344.,  935.,  359.],
          [ 684.,  -40.,  867., ..., -344.,  951.,  359.],
          [ 700.,  -40.,  883., ..., -344.,  967.,  359.]],

         [[ -84.,  -24.,   99., ..., -328.,  183.,  375.],
          [ -68.,  -24.,  115., ..., -328.,  199.,  375.],
          [ -52.,  -24.,  131., ..., -328.,  215.,  375.],
          ...,
          [ 668.,  -24.,  851., ..., -328.,  935.,  375.],
          [ 684.,  -24.,  867., ..., -328.,  951.,  375.],
          [ 700.,  -24.,  883., ..., -328.,  967.,  375.]],

         [[ -84.,   -8.,   99., ..., -312.,  183.,  391.],
          [ -68.,   -8.,  115., ..., -312.,  199.,  391.],
          [ -52.,   -8.,  131., ..., -312.,  215.,  391.],
          ...,
          [ 668.,   -8.,  851., ..., -312.,  935.,  391.],
       

## 2

$$
\begin{cases}
\frac{w_1}{w} = \frac{s}{\sqrt{r}} = \text{round}(\frac{s}{\sqrt{r}})\\
\frac{h_1}{h} = s \sqrt{r} = \text{round}(\frac{w_1}{w} r)
\end{cases}
$$

记

$$
\begin{cases}
S = [s_1, s_1, \cdots, s_m]\\
R = [r_1, r_2, \cdots, r_n]
\end{cases}
$$

则有(下面的运算均是元素级别的元素)：

$$
\begin{cases}
W = (\frac{s_i}{\sqrt{r_j}}) = \frac{S}{\sqrt{R}}\\
H = (s_i \sqrt{r_j}) = W \cdot R
\end{cases}
$$

In [20]:
class AnchorFast(Anchor):
    def __init__(self, base_size, ratios, scales):
        super().__init__(base_size)
        if not isinstance(ratios, (tuple, list)):
            ratios = [ratios]
        if not isinstance(scales, (tuple, list)):
            scales = [scales]
        self.ratios = np.array(ratios)[:, None]
        self.scales = np.array(scales)

    @property
    def W(self):
        '''
        计算 w_1/ w
        '''
        W = self.scales / np.sqrt(self.ratios)
        return np.round(W)

    @property
    def H(self):
        '''
        计算 h_1/ h
        '''
        H = self.W-1 * self.ratios
        return np.round(H)

    @property
    def aspect(self):
        '''
        所有的宽高组合
        '''
        return np.stack([self.W.flatten(), self.H.flatten()], axis=1)

    @property
    def base_anchors(self):
        return self._coordinate(self.aspect, self._whctrs)

    @property
    def anchors(self):
        '''
        获取最终的 base_anchors
        '''
        return self.base_anchors * np.array([self.w, self.h]*2)

In [21]:
scales = [8, 16, 32]  # 尺度，面积比
ratios = [0.5, 1, 2]  # window（滑动窗口） 与锚框的面积的比率（aspect ratios）
base_size = 16
self = AnchorFast(base_size, ratios, scales)

self.anchors

array([[  40.,   48.,  200.,  192.],
       [ -56.,  -48.,  296.,  288.],
       [-232., -224.,  472.,  464.],
       [  64.,   72.,  176.,  168.],
       [   0.,    8.,  240.,  232.],
       [-128., -120.,  368.,  360.],
       [  80.,   96.,  160.,  144.],
       [  40.,   56.,  200.,  184.],
       [ -56.,  -40.,  296.,  280.]])